In [9]:
import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt
from matplotlib.ticker import Formatter
import collections
import glob
import os
import datetime

pd.set_option("max_columns", 200)

for y in ['20200812', '20200813', '20200814', '20200817', '20200821', '20200831', '20200901', '20200902']:
    print('----------------------------------------------------------------')
    print(y)

    re = {}
    for col in ['date', 'data', 'baseline', 'test', 'merge', 'time', 'stock_list']:
        re[col] = []

    readPath = '/mnt/e/new_record_data/' + y + '/***_zs_96_03_day_96data/mdLog_SH_***'
    dataPathLs = np.array(glob.glob(readPath))
    startTm = datetime.datetime.now()
    logSH1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)

    logSH1 = logSH1[["sequenceNo", "StockID", "source", "time", "cum_volume", "cum_amount", "close",
                     "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q", "bid3q",
                     "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q",
                     "ask2q", "ask3q", "ask4q", "ask5q", "openPrice", "numTrades"]]
    logSH1["time"] = logSH1["time"].apply(lambda x: int(x.replace(':', "").replace('.', "")))

    readPath = '/mnt/e/new_record_data/' + y + '/***_zt_88_03_day_88data/mdLog_SH_***'
    dataPathLs = np.array(glob.glob(readPath))
    startTm = datetime.datetime.now()
    logSH2 = pd.read_csv(dataPathLs[0])
    logSH2 = logSH2[["sequenceNo", "StockID", "source", "time", "cum_volume", "cum_amount", "close",
                     "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q", "bid3q",
                     "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q",
                     "ask2q", "ask3q", "ask4q", "ask5q", "openPrice", "numTrades"]]
    logSH2["time"] = logSH2["time"].apply(lambda x: int(x.replace(':', "").replace('.', "")))

    print(datetime.datetime.now() - startTm)
    print('----------------------------------------------------------------')
    print('SH lv2 data:')
    in_dex = [16, 300, 852, 905]
    data1 = logSH2[~logSH2["StockID"].isin(in_dex) & (logSH2["time"] >= 91500000) & (logSH2["time"] <= 150000000)
    & (logSH2['source'] == 23)]
    data2 = logSH1[~logSH1["StockID"].isin(in_dex) & (logSH1["time"] >= 91500000) & (logSH1["time"] <= 150000000) & (
                logSH1['source'] == 13)]
    columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
               "bid2q",
               "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
               "ask4q", "ask5q", "openPrice", "numTrades", "time"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

    n1 = len(data1_1["StockID"].unique())
    n2 = len(data2_1["StockID"].unique())
    print(n1)
    print(n2)
    print(len(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique())))
    print(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique()))

    if n1 != n2:
        sl = list(set(data1_1["StockID"].unique()) & set(data2_1["StockID"].unique()))
        data1_1 = data1_1[data1_1["StockID"].isin(sl)]
        data2_1 = data2_1[data2_1["StockID"].isin(sl)]

    data2_1['cum_amount'] = data2_1['cum_amount'].round(2)
    data1_1['cum_amount'] = data1_1['cum_amount'].round(2)
    data1_1['openPrice'] = data1_1.groupby('StockID')['openPrice'].transform('max')
    data2_1['openPrice'] = data2_1.groupby('StockID')['openPrice'].transform('max')

    data2_1 = data2_1[~data2_1['bid1p'].isnull()]
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    re['date'].append(y)
    re['data'].append('SH lv2 data')
    re['baseline'].append(n1)
    re['test'].append(n2)
    re['merge'].append(len1)
    if (n1 == len1) & (n2 == len1):
        re['time'].append(0)
        re['stock_list'].append(0)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        print("test is not complete:")
        print(test[np.isnan(test["sequenceNo_y"])])
        print(len(test[np.isnan(test["sequenceNo_y"])]) / n1)
        print(len(test[np.isnan(test["sequenceNo_y"])]["time"].unique()))
        print(test[np.isnan(test["sequenceNo_y"])]["time"].unique())
        print(len(test[np.isnan(test["sequenceNo_y"])]["StockID"].unique()))
        print(test[np.isnan(test["sequenceNo_y"])]["StockID"].unique())
        re['time'].append(np.sort(test[np.isnan(test["sequenceNo_y"])]["time"].unique()))
        re['stock_list'].append(np.sort(test[np.isnan(test["sequenceNo_y"])]["StockID"].unique()))
    if (len1 == n2) & (n1 < len1):
        print("baseline is not complete:")
        print(test[np.isnan(test["sequenceNo_x"])])
        print(n2 - n1)
        re['time'].append(np.sort(test[np.isnan(test["sequenceNo_x"])]["time"].unique()))
        re['stock_list'].append(np.sort(test[np.isnan(test["sequenceNo_x"])]["StockID"].unique()))
        print((n2 - n1) / n1)
    del data1
    del data2
    del test
    del data1_1
    del data2_1

    print('----------------------------------------------------------------')
    print('SH lv1 data:')
    in_dex = [16, 300, 852, 905]
    data1 = logSH2[~logSH2["StockID"].isin(in_dex) & (logSH2["time"] <= 150000000) & (
                logSH2['source'] == 22)]
    data2 = logSH1[~logSH1["StockID"].isin(in_dex) & (logSH1["time"] <= 150000000) & (
                logSH1['source'] == 9)]
    columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
               "bid2q",  "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
               "ask4q", "ask5q", "openPrice"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

    n1 = len(data1_1["StockID"].unique())
    n2 = len(data2_1["StockID"].unique())
    print(n1)
    print(n2)
    print(len(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique())))
    print(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique()))

    if n1 != n2:
        sl = list(set(data1_1["StockID"].unique()) & set(data2_1["StockID"].unique()))
        data1_1 = data1_1[data1_1["StockID"].isin(sl)]
        data2_1 = data2_1[data2_1["StockID"].isin(sl)]
    data1_1['cum_amount'] = data1_1['cum_amount'].round(2)
    data2_1['cum_amount'] = data2_1['cum_amount'].round(2)

    data2_1 = data2_1[(data2_1['bid1p'] != 0) | (data2_1['ask1p'] != 0) | (data2_1['cum_volume'] != 0)]
    data1_1 = data1_1[(data1_1['bid1p'] != 0) | (data1_1['ask1p'] != 0) | (data1_1['cum_volume'] != 0)]

    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    re['date'].append(y)
    re['data'].append('SH lv1 data')
    re['baseline'].append(n1)
    re['test'].append(n2)
    re['merge'].append(len1)
    if (n1 == len1) & (n2 == len1):
        re['time'].append(0)
        re['stock_list'].append(0)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        print("test is not complete:")
        print(test[np.isnan(test["sequenceNo_y"])])
        print(len(test[np.isnan(test["sequenceNo_y"])]) / n1)
        print(len(test[np.isnan(test["sequenceNo_y"])]["time_x"].unique()))
        print(test[np.isnan(test["sequenceNo_y"])]["time_x"].unique())
        print(len(test[np.isnan(test["sequenceNo_y"])]["StockID"].unique()))
        print(test[np.isnan(test["sequenceNo_y"])]["StockID"].unique())
        re['time'].append(np.sort(test[np.isnan(test["sequenceNo_y"])]["time_x"].unique()))
        re['stock_list'].append(np.sort(test[np.isnan(test["sequenceNo_y"])]["StockID"].unique()))
    if (len1 == n2) & (n1 < len1):
        print("baseline is not complete:")
        print(test[np.isnan(test["sequenceNo_x"])])
        print(n2 - n1)
        re['time'].append(np.sort(test[np.isnan(test["sequenceNo_x"])]["time_y"].unique()))
        re['stock_list'].append(np.sort(test[np.isnan(test["sequenceNo_x"])]["StockID"].unique()))
        print((n2 - n1) / n1)
    del data1
    del data2
    del test
    del data1_1
    del data2_1

    print('----------------------------------------------------------------')
    print('SH index data:')

    data1 = logSH2[(logSH2["StockID"].isin(in_dex)) & (logSH2["time"] >= 91500000) & (logSH2["time"] <= 150000000)]
    data2 = logSH1[(logSH1["StockID"].isin(in_dex)) & (logSH1["time"] >= 91500000) & (logSH1["time"] <= 150000000) & (logSH1['source'] == 13)]

    columns = ["StockID", "cum_volume", "cum_amount", "close", "openPrice"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

    for cols in ['close', 'openPrice']:
        data1_1[cols] = data1_1[cols].round(4)
        data2_1[cols] = data2_1[cols].round(4)
    for cols in ['cum_amount']:
        data1_1[cols] = data1_1[cols].round(1)
        data2_1[cols] = data2_1[cols].round(1)

    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    re['date'].append(y)
    re['data'].append('SH index data without time column')
    re['baseline'].append(n1)
    re['test'].append(n2)
    re['merge'].append(len1)
    if (n1 == len1) & (n2 == len1):
        re['time'].append(0)
        re['stock_list'].append(0)
    if n2 < len1:
        print("test is not complete:")
        print(test[np.isnan(test["sequenceNo_y"])])
        re['time'].append(np.sort(test[np.isnan(test['sequenceNo_y'])]['time_x'].unique()))
        re['stock_list'].append(np.sort(test[np.isnan(test['sequenceNo_y'])]['StockID'].unique()))
    if (n2 == len1) & (n1 < len1):
        print("baseline is not complete::")
        print(test[np.isnan(test["sequenceNo_x"])])
        re['time'].append(np.sort(test[np.isnan(test["sequenceNo_x"])]['time_y'].unique()))
        re['stock_list'].append(np.sort(test[np.isnan(test['sequenceNo_x'])]['StockID'].unique()))

    del logSH1
    del logSH2
    del data1
    del data2
    del test
    del data1_1
    del data2_1






    readPath = '/mnt/e/new_record_data/' + y + '/***_zs_96_03_day_96data/mdTradeLog_***'
    dataPathLs = np.array(glob.glob(readPath))

    startTm = datetime.datetime.now()
    SH1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)

    readPath = '/mnt/e/new_record_data/' + y + '/***_zt_88_03_day_88data/mdTradeLog_***'
    dataPathLs = np.array(glob.glob(readPath))

    startTm = datetime.datetime.now()
    SH = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)

    print(len(SH["SecurityID"].unique()))
    print(len(SH1["SecurityID"].unique()))
    print(len(set(SH["SecurityID"].unique()) - set(SH1["SecurityID"].unique())))
    print(set(SH["SecurityID"].unique()) - set(SH1["SecurityID"].unique()))

    sl = list(set(SH["SecurityID"].unique()) & set(SH1['SecurityID'].unique()))
    SH = SH[SH["SecurityID"].isin(sl)]
    SH1 = SH1[SH1["SecurityID"].isin(sl)]
    print(len(SH["SecurityID"].unique()))
    print(len(SH1["SecurityID"].unique()))

    print(SH1.columns)

    print('----------------------------------------------------------------')
    print('SH trade data:')

    SH["ExecType"] = 'F'
    SH1["ExecType"] = 'F'
    columns = ["TransactTime", "ApplSeqNum", "SecurityID", "TradePrice", "TradeQty", "TradeMoney", "TradeBSFlag",
               "ExecType",  "BidApplSeqNum", "OfferApplSeqNum"]
    ree = pd.merge(SH, SH1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
    n1 = ree["sequenceNo_x"].count()
    n2 = ree["sequenceNo_y"].count()
    len1 = len(ree)
    print(n1)
    print(n2)
    print(len1)
    re['date'].append(y)
    re['data'].append('SH trade data')
    re['baseline'].append(n1)
    re['test'].append(n2)
    re['merge'].append(len1)
    if (n1 == len1) & (n2 == len1):
        re['time'].append(0)
        re['stock_list'].append(0)
    print("-----------------------------------------------")
    if n2 < len1:
        print("test is not complete:")
        print(ree[np.isnan(ree["sequenceNo_y"])])
        print(len(ree[np.isnan(ree["sequenceNo_y"])]))
        print(np.sort(ree[np.isnan(ree["sequenceNo_y"])]["TransactTime"].unique()))
        print(len(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique()))
        print(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique())
        re['time'].append(np.sort(ree[np.isnan(ree["sequenceNo_y"])]["TransactTime"].unique()))
        re['stock_list'].append(np.sort(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique()))
    if (len1 == n2) & (n1 < len1):
        print("baseline is not complete:")
        print(ree[np.isnan(ree["sequenceNo_x"])])
        print(np.sort(ree[np.isnan(ree["sequenceNo_x"])]["TransactTime"].unique()))
        print(len(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique()))
        print(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique())
        print(n2 - n1)
        re['time'].append(np.sort(ree[np.isnan(ree["sequenceNo_x"])]["TransactTime"].unique()))
        re['stock_list'].append(np.sort(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique()))
    del SH
    del SH1
    del ree


    re = pd.DataFrame(re)
    re.to_csv('/mnt/e/zs_96_03/' + y + '.csv')

----------------------------------------------------------------
20200812
0:00:38.116472
0:00:52.331305
----------------------------------------------------------------
SH lv2 data:
1667
1667
0
set()
7004673
7004673
7004673
-----------------------------------------------
----------------------------------------------------------------
SH lv1 data:
1667
1667
0
set()
6007431
6008429
6008455
-----------------------------------------------
test is not complete:
           index_x  sequenceNo_x  StockID  source_x       time_x  cum_volume  \
3293054  8076205.0   154305606.0   600653      22.0  130443250.0     9668968   
3293057  8076208.0   154305609.0   600685      22.0  130443250.0    23807954   
3293058  8076209.0   154305610.0   600373      22.0  130443250.0     4408028   
3293059  8076210.0   154305611.0   600141      22.0  130443250.0    15538340   
3293062  8076213.0   154305614.0   600520      22.0  130443250.0     4844405   
3293063  8076214.0   154305615.0   600690      22.0  13044

11400
11400
11400


/home/work516/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0:01:17.729477
0:01:29.781127
4770
3898
872
{159973, 200706, 112654, 161810, 200725, 200726, 161815, 127003, 127004, 127005, 127006, 127007, 127008, 161820, 161823, 127011, 127012, 127013, 127014, 127015, 127016, 127017, 127018, 127019, 112676, 161831, 161834, 112682, 161837, 112690, 200761, 159802, 159801, 159803, 159804, 108606, 108607, 108608, 159805, 159806, 159807, 159809, 159810, 159811, 159812, 159813, 159814, 200771, 168002, 108611, 112708, 160805, 106589, 112733, 160806, 160807, 112743, 161903, 166001, 161907, 161908, 166006, 161911, 161912, 166008, 123002, 166009, 123004, 112764, 160812, 123007, 166011, 166012, 123010, 123011, 123012, 123013, 123014, 123015, 160813, 123017, 123018, 166023, 123020, 166024, 123022, 123023, 123024, 123025, 123026, 123027, 123028, 123029, 123030, 123031, 123032, 123033, 123034, 123035, 123036, 123037, 123038, 159901, 123040, 123041, 123042, 123043, 123044, 159908, 159910, 123047, 123048, 123049, 123050, 123051, 123052, 123053, 159913, 123055, 123

[688037 688981 688009 688126 688002 688599 688106 688558 688277 688586
 688311 688268 688012 688003 688069 688138 688312 688027 688180 688396
 688589 688508 688528 688050 688098 688001 688158 688023 688005 688186
 688488 688360 688399 688505 688567 688018 688068 688188]
114
----------------------------------------------------------------
20200813
0:00:33.271790
0:00:48.892092
----------------------------------------------------------------
SH lv2 data:
1667
1667
0
set()
6824662
5606075
6824662
-----------------------------------------------
test is not complete:
          index_x  sequenceNo_x  StockID  source_x       time  cum_volume  \
5606075  11449694     183378219   603040        23  141521000      972064   
5606076  11449695     183378220   600837        23  141521000   178056691   
5606077  11449696     183378282   601330        23  141521000    15795500   
5606078  11449697     183378283   603992        23  141521000     6234960   
5606079  11449698     183378285   603987      

...

0
0
0
-----------------------------------------------
----------------------------------------------------------------
SH lv1 data:
1691
1691
0
set()
5717411
0
5717411
-----------------------------------------------
test is not complete:
          index_x  sequenceNo_x  StockID  source_x     time_x  cum_volume  \
0          184449       2305977   600010        22   91500330           0   
1          184450       2305978   600522        22   91500330           0   
2          184451       2305979   600797        22   91500330           0   
3          184452       2306237   603819        22   91500330           0   
4          184511       2364727   600335        22   91501010           0   
...           ...           ...      ...       ...        ...         ...   
5717406  13905556     226118081   688528        22  145959990      982585   
5717407  13905557     226118082   605188        22  145959990     5595811   
5717408  13905558     226118083   688556        22  145959990     224

0:00:00.041304


/home/work516/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0:02:45.740409
4803
0
4803
{688128, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 688138, 14, 688139, 16, 17, 19, 20, 21, 23, 25, 26, 27, 28, 688157, 30, 31, 32, 688155, 34, 35, 36, 37, 38, 39, 40, 688169, 42, 688166, 688168, 45, 46, 48, 49, 50, 688177, 688180, 688178, 688181, 55, 56, 688185, 58, 59, 60, 61, 62, 63, 688189, 65, 66, 688186, 68, 69, 70, 688196, 688200, 688199, 688202, 688198, 163907, 78, 688208, 88, 89, 90, 688218, 688222, 96, 99, 100, 688229, 688228, 688233, 688256, 688258, 166105, 688266, 688268, 688277, 150, 151, 688278, 153, 155, 156, 157, 158, 159, 688286, 688288, 166, 688298, 688299, 688300, 688309, 688310, 688311, 688312, 688313, 688318, 688321, 688333, 688335, 688336, 688338, 688339, 688357, 688358, 688360, 688363, 688365, 688366, 688368, 688369, 688377, 688379, 688388, 688389, 688393, 164105, 688396, 688398, 688399, 688418, 301, 333, 338, 688466, 688488, 164206, 164208, 164210, 688500, 688505, 688508, 688516, 688518, 688519, 688520, 688521, 400, 401, 402, 403, 404, 688528

0
0
Index(['clockAtArrival', 'sequenceNo', 'exchId', 'securityType',
       '__isRepeated', 'TransactTime', 'ChannelNo', 'ApplSeqNum', 'SecurityID',
       'secid', 'mdSource', 'ExecType', 'TradeBSFlag', '__origTickSeq',
       'TradePrice', 'TradeQty', 'TradeMoney', 'BidApplSeqNum',
       'OfferApplSeqNum'],
      dtype='object')
----------------------------------------------------------------
SH trade data:
0
0
0
-----------------------------------------------


In [10]:
import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt
from matplotlib.ticker import Formatter
import collections
import glob
import os
import datetime

pd.set_option("max_columns", 200)

for y in ['20200901', '20200902', '20200903']:
    print('----------------------------------------------------------------')
    print(y)

    re = {}
    for col in ['date', 'data', 'baseline', 'test', 'merge', 'time', 'stock_list']:
        re[col] = []

    readPath = '/mnt/e/new_record_data/' + y + '/***_zs_96_03_day_96data/mdLog_SH_***'
    dataPathLs = np.array(glob.glob(readPath))
    startTm = datetime.datetime.now()
    logSH1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)

    logSH1 = logSH1[["sequenceNo", "StockID", "source", "time", "cum_volume", "cum_amount", "close",
                     "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q", "bid3q",
                     "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q",
                     "ask2q", "ask3q", "ask4q", "ask5q", "openPrice", "numTrades"]]
    logSH1["time"] = logSH1["time"].apply(lambda x: int(x.replace(':', "").replace('.', "")))

    readPath = '/mnt/e/new_record_data/' + y + '/***_zt_88_03_day_88data/mdLog_SH_***'
    dataPathLs = np.array(glob.glob(readPath))
    startTm = datetime.datetime.now()
    logSH2 = pd.read_csv(dataPathLs[0])
    logSH2 = logSH2[["sequenceNo", "StockID", "source", "time", "cum_volume", "cum_amount", "close",
                     "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q", "bid2q", "bid3q",
                     "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q",
                     "ask2q", "ask3q", "ask4q", "ask5q", "openPrice", "numTrades"]]
    logSH2["time"] = logSH2["time"].apply(lambda x: int(x.replace(':', "").replace('.', "")))

    print(datetime.datetime.now() - startTm)
    print('----------------------------------------------------------------')
    print('SH lv2 data:')
    in_dex = [16, 300, 852, 905]
    data1 = logSH2[~logSH2["StockID"].isin(in_dex) & (logSH2["time"] >= 91500000) & (logSH2["time"] <= 150000000)
    & (logSH2['source'] == 23)]
    data2 = logSH1[~logSH1["StockID"].isin(in_dex) & (logSH1["time"] >= 91500000) & (logSH1["time"] <= 150000000) & (
                logSH1['source'] == 13)]
    columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
               "bid2q",
               "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
               "ask4q", "ask5q", "openPrice", "numTrades", "time"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

    n1 = len(data1_1["StockID"].unique())
    n2 = len(data2_1["StockID"].unique())
    print(n1)
    print(n2)
    print(len(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique())))
    print(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique()))

    if n1 != n2:
        sl = list(set(data1_1["StockID"].unique()) & set(data2_1["StockID"].unique()))
        data1_1 = data1_1[data1_1["StockID"].isin(sl)]
        data2_1 = data2_1[data2_1["StockID"].isin(sl)]

    data2_1['cum_amount'] = data2_1['cum_amount'].round(2)
    data1_1['cum_amount'] = data1_1['cum_amount'].round(2)
    data1_1['openPrice'] = data1_1.groupby('StockID')['openPrice'].transform('max')
    data2_1['openPrice'] = data2_1.groupby('StockID')['openPrice'].transform('max')

    data2_1 = data2_1[~data2_1['bid1p'].isnull()]
    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    re['date'].append(y)
    re['data'].append('SH lv2 data')
    re['baseline'].append(n1)
    re['test'].append(n2)
    re['merge'].append(len1)
    if (n1 == len1) & (n2 == len1):
        re['time'].append(0)
        re['stock_list'].append(0)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        print("test is not complete:")
        print(test[np.isnan(test["sequenceNo_y"])])
        print(len(test[np.isnan(test["sequenceNo_y"])]) / n1)
        print(len(test[np.isnan(test["sequenceNo_y"])]["time"].unique()))
        print(test[np.isnan(test["sequenceNo_y"])]["time"].unique())
        print(len(test[np.isnan(test["sequenceNo_y"])]["StockID"].unique()))
        print(test[np.isnan(test["sequenceNo_y"])]["StockID"].unique())
        re['time'].append(np.sort(test[np.isnan(test["sequenceNo_y"])]["time"].unique()))
        re['stock_list'].append(np.sort(test[np.isnan(test["sequenceNo_y"])]["StockID"].unique()))
    if (len1 == n2) & (n1 < len1):
        print("baseline is not complete:")
        print(test[np.isnan(test["sequenceNo_x"])])
        print(n2 - n1)
        re['time'].append(np.sort(test[np.isnan(test["sequenceNo_x"])]["time"].unique()))
        re['stock_list'].append(np.sort(test[np.isnan(test["sequenceNo_x"])]["StockID"].unique()))
        print((n2 - n1) / n1)
    del data1
    del data2
    del test
    del data1_1
    del data2_1

    print('----------------------------------------------------------------')
    print('SH lv1 data:')
    in_dex = [16, 300, 852, 905]
    data1 = logSH2[~logSH2["StockID"].isin(in_dex) & (logSH2["time"] <= 150000000) & (
                logSH2['source'] == 22)]
    data2 = logSH1[~logSH1["StockID"].isin(in_dex) & (logSH1["time"] <= 150000000) & (
                logSH1['source'] == 9)]
    columns = ["StockID", "cum_volume", "cum_amount", "close", "bid1p", "bid2p", "bid3p", "bid4p", "bid5p", "bid1q",
               "bid2q",  "bid3q", "bid4q", "bid5q", "ask1p", "ask2p", "ask3p", "ask4p", "ask5p", "ask1q", "ask2q", "ask3q",
               "ask4q", "ask5q", "openPrice"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

    n1 = len(data1_1["StockID"].unique())
    n2 = len(data2_1["StockID"].unique())
    print(n1)
    print(n2)
    print(len(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique())))
    print(set(data1_1["StockID"].unique()) - set(data2_1["StockID"].unique()))

    if n1 != n2:
        sl = list(set(data1_1["StockID"].unique()) & set(data2_1["StockID"].unique()))
        data1_1 = data1_1[data1_1["StockID"].isin(sl)]
        data2_1 = data2_1[data2_1["StockID"].isin(sl)]
    data1_1['cum_amount'] = data1_1['cum_amount'].round(2)
    data2_1['cum_amount'] = data2_1['cum_amount'].round(2)

    data2_1 = data2_1[(data2_1['bid1p'] != 0) | (data2_1['ask1p'] != 0) | (data2_1['cum_volume'] != 0)]
    data1_1 = data1_1[(data1_1['bid1p'] != 0) | (data1_1['ask1p'] != 0) | (data1_1['cum_volume'] != 0)]

    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    re['date'].append(y)
    re['data'].append('SH lv1 data')
    re['baseline'].append(n1)
    re['test'].append(n2)
    re['merge'].append(len1)
    if (n1 == len1) & (n2 == len1):
        re['time'].append(0)
        re['stock_list'].append(0)
    print(n1)
    print(n2)
    print(len1)
    print("-----------------------------------------------")
    if n2 < len1:
        print("test is not complete:")
        print(test[np.isnan(test["sequenceNo_y"])])
        print(len(test[np.isnan(test["sequenceNo_y"])]) / n1)
        print(len(test[np.isnan(test["sequenceNo_y"])]["time_y"].unique()))
        print(test[np.isnan(test["sequenceNo_y"])]["time_y"].unique())
        print(len(test[np.isnan(test["sequenceNo_y"])]["StockID"].unique()))
        print(test[np.isnan(test["sequenceNo_y"])]["StockID"].unique())
        re['time'].append(np.sort(test[np.isnan(test["sequenceNo_y"])]["time_y"].unique()))
        re['stock_list'].append(np.sort(test[np.isnan(test["sequenceNo_y"])]["StockID"].unique()))
    if (len1 == n2) & (n1 < len1):
        print("baseline is not complete:")
        print(test[np.isnan(test["sequenceNo_x"])])
        print(n2 - n1)
        re['time'].append(np.sort(test[np.isnan(test["sequenceNo_x"])]["time_y"].unique()))
        re['stock_list'].append(np.sort(test[np.isnan(test["sequenceNo_x"])]["StockID"].unique()))
        print((n2 - n1) / n1)
    del data1
    del data2
    del test
    del data1_1
    del data2_1

    print('----------------------------------------------------------------')
    print('SH index data:')

    data1 = logSH2[(logSH2["StockID"].isin(in_dex)) & (logSH2["time"] >= 91500000) & (logSH2["time"] <= 150000000)]
    data2 = logSH1[(logSH1["StockID"].isin(in_dex)) & (logSH1["time"] >= 91500000) & (logSH1["time"] <= 150000000) & (logSH1['source'] == 13)]

    columns = ["StockID", "cum_volume", "cum_amount", "close", "openPrice"]
    data1_1 = data1.drop_duplicates(subset=columns, keep="first").reset_index()
    data2_1 = data2.drop_duplicates(subset=columns, keep="first").reset_index()

    for cols in ['close', 'openPrice']:
        data1_1[cols] = data1_1[cols].round(4)
        data2_1[cols] = data2_1[cols].round(4)
    for cols in ['cum_amount']:
        data1_1[cols] = data1_1[cols].round(1)
        data2_1[cols] = data2_1[cols].round(1)

    test = pd.merge(data1_1, data2_1, left_on=columns, right_on=columns, how="outer")
    n1 = test["sequenceNo_x"].count()
    n2 = test["sequenceNo_y"].count()
    len1 = len(test)
    print(n1)
    print(n2)
    print(len1)
    re['date'].append(y)
    re['data'].append('SH index data without time column')
    re['baseline'].append(n1)
    re['test'].append(n2)
    re['merge'].append(len1)
    if (n1 == len1) & (n2 == len1):
        re['time'].append(0)
        re['stock_list'].append(0)
    if n2 < len1:
        print("test is not complete:")
        print(test[np.isnan(test["sequenceNo_y"])])
        re['time'].append(np.sort(test[np.isnan(test['sequenceNo_y'])]['time_x'].unique()))
        re['stock_list'].append(np.sort(test[np.isnan(test['sequenceNo_y'])]['StockID'].unique()))
    if (n2 == len1) & (n1 < len1):
        print("baseline is not complete::")
        print(test[np.isnan(test["sequenceNo_x"])])
        re['time'].append(np.sort(test[np.isnan(test["sequenceNo_x"])]['time_y'].unique()))
        re['stock_list'].append(np.sort(test[np.isnan(test['sequenceNo_x'])]['StockID'].unique()))

    del logSH1
    del logSH2
    del data1
    del data2
    del test
    del data1_1
    del data2_1






    readPath = '/mnt/e/new_record_data/' + y + '/***_zs_96_03_day_96data/mdTradeLog_***'
    dataPathLs = np.array(glob.glob(readPath))

    startTm = datetime.datetime.now()
    SH1 = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)

    readPath = '/mnt/e/new_record_data/' + y + '/***_zt_88_03_day_88data/mdTradeLog_***'
    dataPathLs = np.array(glob.glob(readPath))

    startTm = datetime.datetime.now()
    SH = pd.read_csv(dataPathLs[0])
    print(datetime.datetime.now() - startTm)

    print(len(SH["SecurityID"].unique()))
    print(len(SH1["SecurityID"].unique()))
    print(len(set(SH["SecurityID"].unique()) - set(SH1["SecurityID"].unique())))
    print(set(SH["SecurityID"].unique()) - set(SH1["SecurityID"].unique()))

    sl = list(set(SH["SecurityID"].unique()) & set(SH1['SecurityID'].unique()))
    SH = SH[SH["SecurityID"].isin(sl)]
    SH1 = SH1[SH1["SecurityID"].isin(sl)]
    print(len(SH["SecurityID"].unique()))
    print(len(SH1["SecurityID"].unique()))

    print(SH1.columns)

    print('----------------------------------------------------------------')
    print('SH trade data:')

    SH["ExecType"] = 'F'
    SH1["ExecType"] = 'F'
    columns = ["TransactTime", "ApplSeqNum", "SecurityID", "TradePrice", "TradeQty", "TradeMoney", "TradeBSFlag",
               "ExecType",  "BidApplSeqNum", "OfferApplSeqNum"]
    ree = pd.merge(SH, SH1, left_on=columns, right_on=columns, how="outer", validate='one_to_one')
    n1 = ree["sequenceNo_x"].count()
    n2 = ree["sequenceNo_y"].count()
    len1 = len(ree)
    print(n1)
    print(n2)
    print(len1)
    re['date'].append(y)
    re['data'].append('SH trade data')
    re['baseline'].append(n1)
    re['test'].append(n2)
    re['merge'].append(len1)
    if (n1 == len1) & (n2 == len1):
        re['time'].append(0)
        re['stock_list'].append(0)
    print("-----------------------------------------------")
    if n2 < len1:
        print("test is not complete:")
        print(ree[np.isnan(ree["sequenceNo_y"])])
        print(len(ree[np.isnan(ree["sequenceNo_y"])]))
        print(np.sort(ree[np.isnan(ree["sequenceNo_y"])]["TransactTime"].unique()))
        print(len(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique()))
        print(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique())
        re['time'].append(np.sort(ree[np.isnan(ree["sequenceNo_y"])]["TransactTime"].unique()))
        re['stock_list'].append(np.sort(ree[np.isnan(ree["sequenceNo_y"])]["SecurityID"].unique()))
    if (len1 == n2) & (n1 < len1):
        print("baseline is not complete:")
        print(ree[np.isnan(ree["sequenceNo_x"])])
        print(np.sort(ree[np.isnan(ree["sequenceNo_x"])]["TransactTime"].unique()))
        print(len(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique()))
        print(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique())
        print(n2 - n1)
        re['time'].append(np.sort(ree[np.isnan(ree["sequenceNo_x"])]["TransactTime"].unique()))
        re['stock_list'].append(np.sort(ree[np.isnan(ree["sequenceNo_x"])]["SecurityID"].unique()))
    del SH
    del SH1
    del ree


    re = pd.DataFrame(re)
    re.to_csv('/mnt/e/zs_96_03/' + y + '.csv')

----------------------------------------------------------------
20200901
0:01:27.723100
0:00:49.386035
----------------------------------------------------------------
SH lv2 data:
1691
1691
0
set()
6675170
6675170
6675170
-----------------------------------------------
----------------------------------------------------------------
SH lv1 data:
1691
1691
0
set()
5584149
5584927
5584977
-----------------------------------------------
test is not complete:
            index_x  sequenceNo_x  StockID  source_x       time_x  cum_volume  \
3318614   8363289.0   136858632.0   600172      22.0  131833810.0    24758979   
3683834   9226051.0   148022410.0   603056      22.0  133557970.0     6235395   
3683835   9226052.0   148022411.0   600497      22.0  133557970.0    19795232   
3683836   9226053.0   148022412.0   600331      22.0  133557970.0     8343000   
3683837   9226054.0   148022413.0   600355      22.0  133557970.0     4373500   
3683838   9226055.0   148022414.0   600515      22.0

11400
11400
11400
0:00:00.006347


/home/work516/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0:01:09.708874
4806
0
4806
{688128, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 688139, 14, 688138, 16, 17, 19, 20, 21, 23, 25, 26, 27, 28, 688155, 30, 31, 32, 688159, 34, 35, 36, 37, 38, 39, 40, 688165, 42, 688168, 688166, 45, 46, 688169, 48, 49, 50, 688177, 688180, 688181, 688178, 55, 56, 688185, 58, 59, 60, 61, 62, 63, 688188, 65, 66, 688186, 68, 69, 70, 688196, 688200, 688199, 688202, 688198, 163907, 78, 688208, 88, 89, 90, 688218, 688222, 96, 99, 100, 688228, 688229, 688233, 688256, 688258, 166105, 688266, 688268, 688277, 150, 151, 688278, 153, 155, 156, 157, 158, 159, 688286, 688288, 166, 688298, 688299, 688300, 688309, 688310, 688311, 688312, 688313, 688318, 688321, 688333, 688335, 688336, 688338, 688339, 688357, 688358, 688360, 688363, 688365, 688366, 688368, 688369, 688377, 688379, 688388, 688389, 688393, 164105, 688396, 688398, 688399, 688418, 301, 333, 338, 688466, 688488, 164206, 164208, 164210, 688500, 688505, 688508, 688516, 688518, 688519, 688520, 688521, 400, 401, 402, 403, 404

0
0
Index(['clockAtArrival', 'sequenceNo', 'exchId', 'securityType',
       '__isRepeated', 'TransactTime', 'ChannelNo', 'ApplSeqNum', 'SecurityID',
       'secid', 'mdSource', 'ExecType', 'TradeBSFlag', '__origTickSeq',
       'TradePrice', 'TradeQty', 'TradeMoney', 'BidApplSeqNum',
       'OfferApplSeqNum'],
      dtype='object')
----------------------------------------------------------------
SH trade data:
0
0
0
-----------------------------------------------
----------------------------------------------------------------
20200902
0:01:08.249900
0:00:48.534348
----------------------------------------------------------------
SH lv2 data:
1691
1691
0
set()
6811010
6811010
6811010
-----------------------------------------------
----------------------------------------------------------------
SH lv1 data:
1691
1691
0
set()
5717411
5718238
5718238
-----------------------------------------------
baseline is not complete:
         index_x  sequenceNo_x  StockID  source_x  time_x  cum_vol

...

0:01:45.380568
0:01:14.209498
4803
3923
880
{159973, 200706, 112650, 112654, 161810, 112660, 200725, 200726, 161815, 127003, 127004, 127005, 127006, 127007, 127008, 161820, 161823, 127011, 127012, 127013, 127014, 127015, 127016, 127017, 127018, 127019, 127020, 112676, 161831, 112679, 112681, 112682, 161834, 161837, 200761, 159802, 159801, 159803, 159804, 108606, 108607, 159805, 159806, 159807, 159809, 159810, 159811, 159812, 159813, 159814, 159815, 168002, 200771, 159820, 112721, 160805, 106589, 112733, 160806, 160807, 112743, 112747, 161903, 166001, 161907, 161908, 166006, 161911, 161912, 166008, 123002, 166009, 123004, 112764, 166011, 123007, 166012, 166016, 123010, 123011, 123012, 123013, 123014, 123015, 112770, 123017, 123018, 160813, 123020, 112779, 123022, 123023, 123024, 123025, 123026, 123027, 123028, 123029, 123030, 123031, 123032, 123033, 123034, 123035, 123036, 123037, 123038, 159901, 159902, 159903, 123042, 123043, 123044, 123045, 159908, 123047, 123048, 123049, 123050, 123

[688981 688126 688313 688008 688189 688122 688138 688196 688369 688065
 688055 688357 688066 688077 688318 688519 688218 688520 688157 688186
 688599 688002 688336 688012 688488 688030 688393 688396 688029 688185
 688069 688180 688368 688165 688596 688001 688011 688399 688099 688033
 688398 688518 688036]
161
----------------------------------------------------------------
20200903
0:01:12.026690
0:01:18.372963
----------------------------------------------------------------
SH lv2 data:
1691
1691
0
set()
6731118
6731118
6731118
-----------------------------------------------
----------------------------------------------------------------
SH lv1 data:
1691
1691
0
set()
5626635
5627423
5627423
-----------------------------------------------
baseline is not complete:
         index_x  sequenceNo_x  StockID  source_x  time_x  cum_volume  \
5626635      NaN           NaN   600036       NaN     NaN      530300   
5626636      NaN           NaN   600712       NaN     NaN      181700   
5626

/home/work516/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0:02:36.899726
0:01:31.005475
4817
3923
894
{159973, 200706, 112650, 112654, 161810, 112660, 200725, 200726, 161815, 127003, 127004, 127005, 127006, 127007, 127008, 161820, 161823, 127011, 127012, 127013, 127014, 127015, 127016, 127017, 127018, 127019, 127020, 112676, 161831, 112679, 112681, 161834, 112682, 161837, 159801, 200761, 159803, 159804, 159802, 108606, 108607, 159805, 159806, 159807, 159810, 200771, 159811, 159813, 159814, 159812, 159809, 159815, 168002, 159820, 112721, 160805, 106589, 112733, 160806, 160807, 112743, 112747, 161903, 166001, 161907, 161908, 166006, 161911, 161912, 166009, 123002, 166008, 123004, 112764, 160812, 123007, 166011, 166012, 123010, 123011, 123012, 123013, 123014, 123015, 112770, 123017, 123018, 166025, 123020, 112779, 123022, 123023, 123024, 123025, 123026, 123027, 123028, 123029, 123030, 123031, 123032, 123033, 123034, 123035, 123036, 123037, 123038, 159901, 159902, 159903, 159906, 159905, 123044, 123045, 123046, 123047, 123048, 123049, 123050, 123

[688055 688008 688567 688981 688599 688180 688508 688065 688022 688339
 688318 688277 688185 688138 688099 688313 688596 688568 688126 688286
 688158 688088 688085 688393 688309 688028 688519 688520 688556 688466
 688418 688026 688077 688336 688396 688561 688521 688311 688009 688010
 688399 688338 688012 688589]
168
